In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Colab Notebooks/lyrics2')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 239Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 96.0Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 216Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:17, 81.0Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 231Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 92.3Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 18.5Mit/s]                                                      


In [ ]:
file_name = "nursery_rhymes.txt"

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=("dataset/" + file_name),
              model_name='355M',
              steps=2000,
              restore_from='fresh',
              run_name='run6',
              print_every=10,
              sample_every=500,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


dataset has 14748 tokens
Training...
[10 | 17.84] loss=2.64 avg=2.64
[20 | 26.67] loss=2.51 avg=2.58
[30 | 35.50] loss=1.12 avg=2.09
[40 | 44.33] loss=0.58 avg=1.70
[50 | 53.16] loss=0.06 avg=1.37
[60 | 61.98] loss=0.05 avg=1.14
[70 | 70.81] loss=0.06 avg=0.98
[80 | 79.64] loss=0.02 avg=0.86
[90 | 88.47] loss=0.01 avg=0.76
[100 | 97.29] loss=0.07 avg=0.69
[110 | 106.11] loss=0.02 avg=0.62
[120 | 114.93] loss=0.02 avg=0.57
[130 | 123.76] loss=0.03 avg=0.53
[140 | 132.59] loss=0.02 avg=0.49
[150 | 141.42] loss=0.05 avg=0.46
[160 | 150.23] loss=0.03 avg=0.43
[170 | 159.06] loss=0.04 avg=0.40
[180 | 167.90] loss=0.03 avg=0.38
[190 | 176.72] loss=0.07 avg=0.36
[200 | 185.55] loss=0.01 avg=0.34
[210 | 194.38] loss=0.04 avg=0.33
[220 | 203.20] loss=0.02 avg=0.31
[230 | 212.02] loss=0.06 avg=0.30
[240 | 220.85] loss=0.03 avg=0.29
[250 | 229.68] loss=0.03 avg=0.28
[260 | 238.51] loss=0.02 avg=0.26
[270 | 247.33] loss=0.01 avg=0.25
[280 | 256.15] loss=0.02 avg=0.24
[290 | 264.98] loss=0.01 avg=0

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='rap-1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run6')

In [ ]:
#import tensorflow as tf
sess = gpt2.start_tf_sess()
#tf.reset_default_graph()
gpt2.load_gpt2(sess, run_name='run6')
#gpt2.generate(sess, run_name='rap-1')

Loading checkpoint checkpoint/run6/model-500
INFO:tensorflow:Restoring parameters from checkpoint/run6/model-500


In [ ]:
with open(("dataset/" + file_name), 'r') as f:
  dset = f.read()

import collections, nltk
# we first tokenize the text corpus
nltk.download('punkt')
tokens = nltk.word_tokenize(dset)

def unigram(tokens):
  model = collections.defaultdict(lambda: 0.1)
  #print(type(model))
  for f in tokens:
    try:
      model[f] += 1
    except KeyError:
      model [f] = 1
      continue
  N = float(sum(model.values()))
  for word in model:
    model[word] = model[word]/N
  return model

#computes perplexity of the unigram model on a testset    
def perplexity(testset, model):
  testset = testset.split()
  perplexity = 1
  N = 0
  for word in testset:
    N += 1
    perplexity = perplexity * (1/model[word])
  perplexity = pow(perplexity, 1/float(N)) 
  return perplexity

model2 = unigram(tokens)
print(perplexity("nothing", model2))

"""
ppls = []

for i in range(40):
  snippet1 = gpt2.generate(sess, run_name="rap-1", return_as_list=True)[0]
  snippet1 = snippet1.split("\n")[0]
  print(snippet1)
  ppl = perplexity(snippet1, model2)
  print("i:", i)
  print(ppl)
  ppls.append(ppl)
  print("avg:", sum(ppls)/(len(ppls) + 1))

print(sum(ppls)/len(ppls))"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2011.9392789353158


'\nppls = []\n\nfor i in range(40):\n  snippet1 = gpt2.generate(sess, run_name="rap-1", return_as_list=True)[0]\n  snippet1 = snippet1.split("\n")[0]\n  print(snippet1)\n  ppl = perplexity(snippet1, model2)\n  print("i:", i)\n  print(ppl)\n  ppls.append(ppl)\n  print("avg:", sum(ppls)/(len(ppls) + 1))\n\nprint(sum(ppls)/len(ppls))'

In [ ]:
# next word prediction
import random


def rm_from_list(somelist):
    badlines = ["", "\n"]
    for i in badlines:
      somelist = [x for x in somelist if x not in badlines]
    return somelist
    
with open(("dataset/" + file_name), 'r') as f:
  dset = f.read()
dset = rm_from_list(dset.split("\n"))

num_correct = 0

for i in range(50):
  random_line = dset[random.randint(0, len(dset))]
  words = random_line.split(" ")
  split_at = int(len(words)/2)
  #print(words)
  prevwords = " ".join(words[:split_at])
  correct_word = words[split_at]
  print("prevwords is:", prevwords, "- correct_word is:", correct_word)

  snippet1 = gpt2.generate(sess, run_name="run6", return_as_list=True, prefix=prevwords)[0].split("\n")
  #print(snippet1)
  #print(snippet1[:1])
  if snippet1[0] == prevwords:
    print("thought end of line")
  else:
    nextword = snippet1[0].replace(prevwords, "").split(" ")
    #print("nw", nextword)
    if nextword[0] == " " or nextword[0] == "":
      nextword = nextword[1]
    else:
      nextword = nextword[0]
    #print("nextword is:", nextword)
    print("WE PREDICTED THE CORRECT WORD: ", nextword == correct_word)
    if nextword == correct_word:
      num_correct += 1
    print(num_correct/(i+1))

prevwords is:     - correct_word is: R,
WE PREDICTED THE CORRECT WORD:  False
0.0
prevwords is:     I - correct_word is: put
WE PREDICTED THE CORRECT WORD:  False
0.0
prevwords is:      - correct_word is: 
WE PREDICTED THE CORRECT WORD:  True
0.3333333333333333
prevwords is:     - correct_word is: What
WE PREDICTED THE CORRECT WORD:  False
0.25
prevwords is:       And - correct_word is: he'd
WE PREDICTED THE CORRECT WORD:  False
0.2
prevwords is:     The lion and - correct_word is: the
WE PREDICTED THE CORRECT WORD:  True
0.3333333333333333
prevwords is:     - correct_word is: 
WE PREDICTED THE CORRECT WORD:  True
0.42857142857142855
prevwords is:      - correct_word is: 
WE PREDICTED THE CORRECT WORD:  False
0.375
prevwords is:     - correct_word is: 
WE PREDICTED THE CORRECT WORD:  True
0.4444444444444444
prevwords is:     He - correct_word is: stuck
WE PREDICTED THE CORRECT WORD:  False
0.4
prevwords is: DANTY - correct_word is: BABY
WE PREDICTED THE CORRECT WORD:  False
0.363636363

KeyboardInterrupt: ignored

In [ ]:
snippet1 = gpt2.generate(sess, run_name="rap-1", return_as_list=True, temperature=0.5)[0].split("\n")

def rm_from_list(somelist):
    badlines = ["", "\n"]
    for i in badlines:
      somelist = [x for x in somelist if x not in badlines]
    return somelist
    
with open(("dataset/" + file_name), 'r') as f:
  dset = f.read()
dset = rm_from_list(dset.split("\n"))
snippet1 = rm_from_list(snippet1)

#print(dset)
#print(snippet1)

def line_overlap(snippet):
  snippet = rm_from_list(snippet)
  overlapnum = len(list(set(dset) & set(snippet)))
  lesser = min(len(dset), len(snippet))
  return overlapnum/lesser


def wordFreq(snippet):
  from collections import Counter
  ds = "\n".join(dset).split(" ")
  sp = "\n".join(snippet).split(" ")
  c1 = Counter(ds).most_common(20)
  c2 = Counter(sp).most_common(20)
  cw1 = [i[0] for i in c1]
  cw2 = [i[0] for i in c2]
  return len(list(set(cw1) & set(cw2)))/20

print(line_overlap(snippet1))
print(wordFreq(snippet1))

In [ ]:
gpt2.generate(sess)

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=1,
                      nsamples=100,
                      batch_size=20,
                      run_name="run6"
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

In [ ]:
# run if errors
#!kill -9 -1